In [39]:
import requests
import json
import math

def color_text(text, color_code):
    """Apply ANSI color codes to the given text."""
    return f"\033[{color_code}m{text}\033[0m"

COLORS = {
    "red": "91", "green": "92", "yellow": "93", "light_purple": "94",
    "purple": "95", "cyan": "96", "light_gray": "97", "black": "98", "bold": "1"
}

def round_nearest(x, base=50):
    """Round the number to the nearest base."""
    return int(math.ceil(x / base) * base)

API_ENDPOINTS = {
    "oc": "https://www.nseindia.com/option-chain",
    "bnf": "https://www.nseindia.com/api/option-chain-indices?symbol=BANKNIFTY",
    "nf": "https://www.nseindia.com/api/option-chain-indices?symbol=NIFTY",
    "indices": "https://www.nseindia.com/api/allIndices"
}

HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.149 Safari/537.36',
    'Accept-Language': 'en,gu;q=0.9,hi;q=0.8',
    'Accept-Encoding': 'gzip, deflate, br'
}

sess = requests.Session()
cookies = {}

def set_cookie():
    """Retrieve and set cookies for the session."""
    global cookies
    try:
        response = sess.get(API_ENDPOINTS['oc'], headers=HEADERS, timeout=5)
        cookies = dict(response.cookies)
    except requests.RequestException as e:
        print(f"Error setting cookies: {e}")

def get_data(url):
    """Retrieve data from the given URL."""
    set_cookie()
    try:
        response = sess.get(url, headers=HEADERS, timeout=5, cookies=cookies)
        if response.status_code == 401:
            set_cookie()
            response = sess.get(url, headers=HEADERS, timeout=5, cookies=cookies)
        if response.status_code == 200:
            return response.text
    except requests.RequestException as e:
        print(f"Error fetching data: {e}")
    return None

def highest_oi(option_type, num, step, nearest, url):
    """Find the strike price with the highest open interest for the given option type."""
    strike = nearest - (step * num)
    start_strike = strike
    response_text = get_data(url)
    if response_text:
        data = json.loads(response_text)
        curr_expiry_date = data["records"]["expiryDates"][0]
        max_oi = 0
        max_oi_strike = 0

        for item in data['records']['data']:
            if item["expiryDate"] == curr_expiry_date:
                if start_strike <= item["strikePrice"] < start_strike + (step * num * 2):
                    oi = item[option_type]["openInterest"]
                    if oi > max_oi:
                        max_oi = oi
                        max_oi_strike = item["strikePrice"]

        return max_oi_strike
    else:
        print("Failed to retrieve data.")
        return None

def set_header():
    """Set the header information for Nifty and Bank Nifty."""
    global bnf_ul, nf_ul, bnf_nearest, nf_nearest
    response_text = get_data(API_ENDPOINTS['indices'])
    if response_text:
        data = json.loads(response_text)

        for index in data["data"]:
            if index["index"] == "NIFTY 50":
                nf_ul = index["last"]
                print("Nifty")
            if index["index"] == "NIFTY BANK":
                bnf_ul = index["last"]
                print("Bank Nifty")

        bnf_nearest = round_nearest(bnf_ul, 100)
        nf_nearest = round_nearest(nf_ul, 50)
    else:
        print("Failed to retrieve index data.")

def print_header(index, ul, nearest):
    """Print header information for an index."""
    print(color_text(index.ljust(12, " ") + "=> ", COLORS["purple"]) +
          color_text(" Last Price: ", COLORS["light_purple"]) +
          color_text(str(ul), COLORS["bold"]) +
          color_text(" Nearest Strike: ", COLORS["light_purple"]) +
          color_text(str(nearest), COLORS["bold"]))

def print_hr():
    """Print a horizontal rule."""
    print(color_text("|".rjust(70, "-"), COLORS["yellow"]))

def print_oi(num, step, nearest, url):
    """Print the open interest for options."""
    strike = nearest - (step * num)
    start_strike = strike
    response_text = get_data(url)
    if response_text:
        data = json.loads(response_text)
        curr_expiry_date = data["records"]["expiryDates"][0]

        for item in data['records']['data']:
            if item["expiryDate"] == curr_expiry_date:
                if start_strike <= item["strikePrice"] < start_strike + (step * num * 2):
                    ce_oi = item["CE"]["openInterest"] if "CE" in item else "NA"
                    pe_oi = item["PE"]["openInterest"] if "PE" in item else "NA"
                    print(color_text(str(item["strikePrice"]), COLORS["cyan"]) + color_text(" CE ", COLORS["green"]) +
                          "[" + color_text(str(ce_oi).rjust(10, " "), COLORS["bold"]) + "]" +
                          color_text(" PE ", COLORS["red"]) + "[ " + color_text(str(pe_oi).rjust(10, " "), COLORS["bold"]) + " ]")
                    strike += step
    else:
        print("Failed to retrieve options data.")

# Main execution logic
set_header()
print("\033c")  # Clear console (Note: This may not work in some environments)
print_hr()
print_header("Nifty", nf_ul, nf_nearest)
print_hr()
print_oi(10, 50, nf_nearest, API_ENDPOINTS['nf'])
print_hr()
print_header("Bank Nifty", bnf_ul, bnf_nearest)
print_hr()
print_oi(10, 100, bnf_nearest, API_ENDPOINTS['bnf'])
print_hr()

nf_highestoi_CE = highest_oi("CE", 10, 50, nf_nearest, API_ENDPOINTS['nf'])
nf_highestoi_PE = highest_oi("PE", 10, 50, nf_nearest, API_ENDPOINTS['nf'])
bnf_highestoi_CE = highest_oi("CE", 10, 100, bnf_nearest, API_ENDPOINTS['bnf'])
bnf_highestoi_PE = highest_oi("PE", 10, 100, bnf_nearest, API_ENDPOINTS['bnf'])

print(color_text("Major Support in Nifty: ", COLORS["cyan"]) + str(nf_highestoi_CE))
print(color_text("Major Resistance in Nifty: ", COLORS["cyan"]) + str(nf_highestoi_PE))
print(color_text("Major Support in Bank Nifty: ", COLORS["purple"]) + str(bnf_highestoi_CE))
print(color_text("Major Resistance in Bank Nifty: ", COLORS["purple"]) + str(bnf_highestoi_PE))



Nifty
Bank Nifty
c
---------------------------------------------------------------------|
Nifty       =>  Last Price: 21352.6 Nearest Strike: 21400
---------------------------------------------------------------------|
20900 CE [      1215] PE [      81249 ]
20950 CE [       628] PE [      39534 ]
21000 CE [     10916] PE [     136131 ]
21050 CE [      1908] PE [      24841 ]
21100 CE [      5374] PE [      57633 ]
21150 CE [     11331] PE [      52507 ]
21200 CE [     48456] PE [     123423 ]
21250 CE [     71022] PE [     212200 ]
21300 CE [    298258] PE [     182704 ]
21350 CE [    202131] PE [      77507 ]
21400 CE [    201719] PE [      58205 ]
21450 CE [     89859] PE [      31900 ]
21500 CE [    140067] PE [      29875 ]
21550 CE [     61981] PE [       4738 ]
21600 CE [    105212] PE [      16193 ]
21650 CE [     69591] PE [       4000 ]
21700 CE [    126777] PE [      22694 ]
21750 CE [     51532] PE [      11735 ]
21800 CE [    123763] PE [      13992 ]
21850 CE [     40865